In [0]:
#Make sure tf 2.x is installed
#!pip install -U tensorflow --quiet

In [152]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
import tensorflow as tf
tf.__version__

'1.15.0'

# 1. Read the dataset

In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
bank_df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/R6/Project1/Churn.csv")

In [156]:
bank_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# 2. Drop the columns which are unique for all users like IDs (2.5 points)

In [0]:
bank_df.drop(['RowNumber','CustomerId','Surname'],inplace=True, axis=1)

In [158]:
bank_df.head()

# Geography and Gender needs to be converted into numerical values using Label Encoding

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 

bank_df['Geography']= label_encoder.fit_transform(bank_df['Geography']) 
label_encoder = preprocessing.LabelEncoder()
bank_df['Gender']= label_encoder.fit_transform(bank_df['Gender']) 

In [160]:
print(bank_df['Geography'].unique())

[0 2 1]


In [161]:
print(bank_df['Gender'].unique())

[0 1]


# 3. Distinguish the feature and target set (2.5 points)

In [0]:
# Dependent Variables, features
X = bank_df.drop('Exited', axis=1)

# In Dependent Variables, target
Y = bank_df['Exited']

# 4. Divide the data set into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test_old = train_test_split(X, Y, test_size=0.2)

# 5. Normalize the train and test data (2.5 points)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_new=scaler.fit_transform(x_train)
x_test_new=scaler.fit_transform(x_test)

In [165]:
x_train_new.shape

(8000, 10)

In [166]:
x_test_new.shape

(2000, 10)

In [167]:
y_train.shape

(8000,)

In [168]:
y_train.unique()

array([0, 1])

# 6. Initialize & build the model (10 points)

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

In [0]:
#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(10,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

In [0]:
#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(2, activation='softmax'))

#Comile the model
# Another option is binary_crossentropy
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [173]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

print(x_train_new.shape)


(8000, 10)
(8000,)
(2000, 10)
(2000, 2)
(8000, 10)


In [174]:
y_train

1050    0
3363    0
9274    0
1619    0
6328    0
       ..
3376    1
1308    0
8395    0
459     0
3263    1
Name: Exited, Length: 8000, dtype: int64

In [0]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test_old, num_classes=2)

In [176]:
model.fit(x_train_new, y_train, epochs=10)

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 101us/sample - loss: 0.6631 - acc: 0.7055
Epoch 2/10
8000/8000 [==============================] - 1s 105us/sample - loss: 0.5090 - acc: 0.7949
Epoch 3/10
8000/8000 [==============================] - 1s 86us/sample - loss: 0.5038 - acc: 0.7949
Epoch 4/10
8000/8000 [==============================] - 1s 92us/sample - loss: 0.5023 - acc: 0.7949
Epoch 5/10
8000/8000 [==============================] - 1s 91us/sample - loss: 0.5002 - acc: 0.7949
Epoch 6/10
8000/8000 [==============================] - 1s 86us/sample - loss: 0.4974 - acc: 0.7949
Epoch 7/10
8000/8000 [==============================] - 1s 86us/sample - loss: 0.4942 - acc: 0.7949
Epoch 8/10
8000/8000 [==============================] - 1s 87us/sample - loss: 0.4905 - acc: 0.7949
Epoch 9/10
8000/8000 [==============================] - 1s 95us/sample - loss: 0.4861 - acc: 0.7956
Epoch 10/10
8000/8000 [==============================] - 1s 86us/sample - lo

In [0]:
predict=model.predict(x_train)

In [178]:
predict

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [179]:
loss_and_metrics = model.evaluate(x_train_new, y_train)
print(loss_and_metrics)

8000/8000 [==============================] - 1s 68us/sample - loss: 0.4762 - acc: 0.7959
[0.4762248303890228, 0.795875]


Accuracy is coming as 79.58%

# 7. Optimize the model (5 points)

Optimization cycle 1

In [0]:
#Initialize Sequential model
model2 = tf.keras.models.Sequential()


In [0]:
model2.add(tf.keras.layers.BatchNormalization())
#Add Dense layer for prediction - Keras declares weights and bias automatically
model2.add(tf.keras.layers.Dense(100, input_shape=(10,),activation='relu', name='Layer_1'))

#Normalize the data
model2.add(tf.keras.layers.BatchNormalization())

model2.add(tf.keras.layers.Dense(50, activation='relu', name='Layer_2'))

model2.add(tf.keras.layers.BatchNormalization())

model2.add(tf.keras.layers.Dense(10, activation='relu', name='Layer_3'))


#Add Dense Layer which provides 10 Outputs after applying softmax
model2.add(tf.keras.layers.Dense(2, activation='softmax'))

#Comile the model
# Another option is binary_crossentropy
model2.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [182]:
model2.fit(x_train_new, y_train, epochs=10)

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 171us/sample - loss: 0.4837 - acc: 0.7703
Epoch 2/10
8000/8000 [==============================] - 1s 155us/sample - loss: 0.3813 - acc: 0.8431
Epoch 3/10
8000/8000 [==============================] - 1s 142us/sample - loss: 0.3749 - acc: 0.8425
Epoch 4/10
8000/8000 [==============================] - 1s 147us/sample - loss: 0.3595 - acc: 0.8522
Epoch 5/10
8000/8000 [==============================] - 1s 142us/sample - loss: 0.3549 - acc: 0.8514
Epoch 6/10
8000/8000 [==============================] - 1s 150us/sample - loss: 0.3535 - acc: 0.8519
Epoch 7/10
8000/8000 [==============================] - 1s 147us/sample - loss: 0.3457 - acc: 0.8561
Epoch 8/10
8000/8000 [==============================] - 1s 144us/sample - loss: 0.3448 - acc: 0.8533
Epoch 9/10
8000/8000 [==============================] - 1s 139us/sample - loss: 0.3410 - acc: 0.8562
Epoch 10/10
8000/8000 [==============================] - 1s 140us/sam

In [183]:
loss_and_metrics2 = model2.evaluate(x_train.values, y_train)
print(loss_and_metrics2)

8000/8000 [==============================] - 1s 71us/sample - loss: 147470.2412 - acc: 0.2051
[147470.24125, 0.205125]


Optimization Cycle 2

In [184]:
#Initialize Sequential model
model3 = tf.keras.models.Sequential()

model3.add(tf.keras.layers.BatchNormalization())

#Add Dense layer for prediction - Keras declares weights and bias automatically
model3.add(tf.keras.layers.Dense(200, input_shape=(10,), name='Layer_1',kernel_initializer='he_normal'))
model3.add(tf.keras.layers.LeakyReLU())
model3.add(tf.keras.layers.BatchNormalization())
model3.add(tf.keras.layers.Dropout(0.25))

model3.add(tf.keras.layers.Dense(100, name='Layer_2', kernel_initializer='he_normal'))
model3.add(tf.keras.layers.LeakyReLU())
model3.add(tf.keras.layers.BatchNormalization())

model3.add(tf.keras.layers.Dense(50, name='Layer_3', kernel_initializer='he_normal'))
model3.add(tf.keras.layers.LeakyReLU())
model3.add(tf.keras.layers.BatchNormalization())

model3.add(tf.keras.layers.Dense(2, activation='softmax'))

#Comile the model
# Another option is binary_crossentropy
model3.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])
model3.fit(x_train.values, y_train, epochs=100)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 2s 205us/sample - loss: 0.5029 - acc: 0.7661
Epoch 2/100
8000/8000 [==============================] - 1s 171us/sample - loss: 0.4024 - acc: 0.8285
Epoch 3/100
8000/8000 [==============================] - 1s 175us/sample - loss: 0.3916 - acc: 0.8365
Epoch 4/100
8000/8000 [==============================] - 1s 174us/sample - loss: 0.3765 - acc: 0.8441
Epoch 5/100
8000/8000 [==============================] - 1s 170us/sample - loss: 0.3731 - acc: 0.8451
Epoch 6/100
8000/8000 [==============================] - 1s 185us/sample - loss: 0.3710 - acc: 0.8443
Epoch 7/100
8000/8000 [==============================] - 2s 195us/sample - loss: 0.3696 - acc: 0.8429
Epoch 8/100
8000/8000 [==============================] - 2s 188us/sample - loss: 0.3678 - acc: 0.8471
Epoch 9/100
8000/8000 [==============================] - 1s 184us/sample - loss: 0.3626 - acc: 0.8468
Epoch 10/100
8000/8000 [==============================] - 1s

In [185]:
loss_and_metrics3 = model3.evaluate(x_train.values, y_train)
print(loss_and_metrics3)

8000/8000 [==============================] - 1s 85us/sample - loss: 0.3029 - acc: 0.8733
[0.30288825619220733, 0.87325]


Since the data is very less, it becomes very hard to improve the model accuracy beyond this point. Please suggest if anything else can also be done to further improve the accuracy

# 9. Predict the results using 0.5 as a threshold (5 points)

In [186]:
y_pred = model3.predict(x_test_new, verbose=0)
y_pred_classes = model3.predict_classes(x_test_new, verbose=0)
print(y_pred)

[[0.5113697  0.4886303 ]
 [0.9311421  0.06885789]
 [0.2971903  0.7028097 ]
 ...
 [0.33466354 0.66533643]
 [0.9248479  0.07515208]
 [0.8074735  0.19252647]]


In [0]:
y_pred2 = [1 if x[1] >= .5 else 0 for x in y_pred]

In [188]:
print(np.transpose(np.matrix(y_pred2)))

[[0]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]


# 10. Print the Accuracy score and confusion matrix (2.5 points)

In [0]:
_,acc=model2.evaluate(x_test_new, np.asarray(y_pred2,order='F').T, verbose=0)

In [190]:
print("Accuracy is ",acc*100, "%")

Accuracy is  87.55000233650208 %


In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
y_pred_arr=np.array(y_pred2)

In [0]:
y_pred_arr2= np.transpose(np.matrix(y_pred_arr))
#y_pred_arr.reshape((1, 2000), order='F')

In [195]:
print(np.array(y_pred_arr2))

[[0]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]


In [196]:
print("x_train_new:",x_train_new.shape)
print("x_test_new:",x_test_new.shape)
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)
print("y_test_old:",y_test_old.shape)
print("y_pred_arr2:",y_pred_arr2.shape)
print("y_pred_arr:",y_pred_arr.shape)
print("y_pred_arr:transpose",y_pred_arr.transpose().shape)

x_train_new: (8000, 10)
x_test_new: (2000, 10)
y_train: (8000, 2)
y_test: (2000, 2)
y_test_old: (2000,)
y_pred_arr2: (2000, 1)
y_pred_arr: (2000,)
y_pred_arr:transpose (2000,)


In [0]:
# Converting 1D row array to 1D column array
y_pred_classes2 = y_pred_classes.flatten()

In [198]:
matrix = confusion_matrix(y_test_old, y_pred_classes2.transpose())
print(matrix)

[[1316  288]
 [ 247  149]]


# Please suggest what can be done to further improve the accuracy